## Imports

In [ ]:
include("Rocket_mv.jl")
include("RocketHelperFunctions_mv.jl")
using MLJ
using MLJLinearModels
using ARFFFiles,DataFrames,CategoricalArrays
using Catch22

## Utility Functions for data import and regression classifier training

In [ ]:
"""
Stores multivariate data in an array with structure [samples, dimensions, timepoints].
### Parameters
    1. data : DataFrame
        -a collection of imported single channel files
    2. dims : int
        -the number of channels/dimensions per sample
    3. leng : int
        -the number of timepoints in each sample.

### Returns
    - An array in format [samples, dimensions, timepoints] for use with multivariate rocket transform functions
"""
function prep_data(data, dims, leng) 
    cells = size(data[1])[1]
    X = zeros((cells, dims, leng))
    for i in 1:cells
        for j in 1:dims
            X[i,j,:] = Array(data[j][i,1:leng])
        end
    end
    return X
end

#this should work for any problem (single or multichannel input, multiclass or binary classification), based on MLJLinearModels implementation
"""
Builds a tuned logistic regression classifier model regularized with an l1 penalty.

### Parameters:
    1. X : DataFrame
        -a dataframe containing kernel-transformed training data with labels
    2. Y : CategoricalArray
        -labels for each training example
    3. l : int
        -lower bound for lambda
    4. u : int
        -upper bound for lambda
    5. num_models : int
        -number of models to compare for hyperparameter tuning
    6. num_folds : int
        -number of cross-validation folds for hyperparameter tuning
    
### Returns:
    -a MLJ logistic regression classifier model trained on (X, Y)
"""
function regression_fitting(X::DataFrame, Y::CategoricalArray,l::Float64, u::Float64, num_models::Int, num_folds::Int)
    
    log_classifier = LogisticClassifier(penalty=:l1)

    lambda_range = range(log_classifier, :lambda, lower=l, upper=u) #Defining range object for hyperparameter tuning
    self_tuning_model= TunedModel(model=log_classifier,
        resampling = CV(nfolds=num_folds, rng=1234),#rng seeds to standardize
        tuning = RandomSearch(), #change to Grid() for grid search.
        ranges = lambda_range,
        n=num_models);


    mach = machine(self_tuning_model, X,CategoricalArray(Y))
    
    fit!(mach) #Fit model! NOTE defaults to minimizing cross-entropy loss for training.
    return mach
end

"""
A function to import training and testing data from a data folder (via the links in each example).

### Parameters:
    1. ds_string : String
        -the file name prefix for each data file.
    2. n_dims : int
        -the number of channels per sample to read
    3. starts_at : int
        -the column at which timepoint data begins in each file 
    4. train : boolean
        -whether the import is for training or testing data
    5. standardize : boolean
        -whether to standardize each sample (example - example_mean)/(example_sd)
    6. series_length : int
        -how many timepoints to inclue for each sample
    7. label_column : int
        -the column containing label data
### Returns:
    -A tuple (g, labels_train) containing the desired multivariate data array g and the labels for this dataset.
"""
function import_data(ds_string::String, n_dims::Int64, starts_at::Int64 = 1, train::Bool = true, standardize::Bool = false, series_length::Int64 = 144, label_column::Int64 = 145)
    channels_arr = []
    train_string = ""
    if train == true
        train_string = "TRAIN"
    else
        train_string = "TEST"
    end
        
    for i in range(starts_at, n_dims)
        df = ARFFFiles.load(DataFrame, "../data/"*ds_string * string(i)*"_"*train_string*".arff")
        push!(channels_arr,df)
    end
    labelsn = channels_arr[1][:,label_column]
    g = prep_data(channels_arr, n_dims, series_length);
    if standardize
        g, = normalize_data_matrix_mv(g)
    end
    return g, labels
end

# Examples of Multivariate Time Series applications of ROCKET.

For mulivariate time series, we take the approach of generating rectangular kernels in a manner similar to the single channel case. Padding remains the same for all channels, as does dilation -- this limits the use of kernels at multiple timescales per channel. These limitations are necessary for computational efficiency.

Kernels for multivariable data are generated by constant dilation, padding, and bias per channel, with normally-distributed weights.  The kernels cover significantly less of the search space as the channel count increases, and thus results become much less stable (see: https://en.wikipedia.org/wiki/Curse_of_dimensionality).


We use examples from the Time Series Machine Learning Website (http://www.timeseriesclassification.com/index.php) below.

## Example 1: ArticularyWordRecognitionDimension
This dataset has 9 channels and 25 classes, making it a demonstration of the disutility of ROCKET for data with an excess of input channels.

### Load and standardize data

In [ ]:
#import data
ds_string = "ArticularyWordRecognitionDimension"
n_dims = 9
starts_at = 1
series_length = 144

X_train, Y_train = import_data(ds_string, n_dims, starts_at, true, true, series_length,145)
X_test, Y_test = import_data(ds_string, n_dims, starts_at, false, true, series_length,145)

### Kernel generation and ROCKET Transform

In [ ]:

#Generate ROCKET kernels
kers = 10000
K = generate_kernels_mv(series_length,n_dims,kers);

#generate column names for each kernel -- for tracking relevant kernels later.
K_symbols = cat([Symbol(k.name,"max") for k in K], [Symbol(k.name,"ppv") for k in K],dims=1);

#transform data, store in dataframe with column labels for each kernel feature.
X_train_transform = Rocket_transform_mv(X_train,K)[1];
df_train = DataFrame(X_train_transform, K_symbols);
X_test_transform = Rocket_transform_mv(X_test,K)[1];
df_test = DataFrame(X_test_transform, K_symbols);




In [ ]:
#call regression fitting function on training data
mach = regression_fitting(df_train, Y_train, 0.0, 1.0, 10, 5)

In [ ]:
#prediction
y_pred = predict_mode(mach,df_test)
y_train_pred = predict_mode(mach, df_train)

#confusion matrices
MLJ.confusion_matrix(y_pred, Y_test)
#MLJ.confusion_matrix(y_train_pred, Y_train)


In [ ]:
sum(y_pred .== Y_test)/length(y_pred)

In [ ]:
params=fitted_params(mach)
best_coefs=params.best_fitted_params.coefs #coefs of model with lowest log_loss
best_coef_sum_vals = [c[2][1] for c in best_coefs]
sum(best_coef_sum_vals.== 0)
r = best_coef_sum_vals[best_coef_sum_vals .!= 0]
r

# Example 1a: Augmenting ROCKET with catch22

catch22 {`Lubba2019`, https://doi.org/10.1007/s10618-019-00647-x) is a collection of discriminative time series features generated from the highly comparative time-series analysis (hctsa https://zenodo.org/badge/latestdoi/10790340) features.  These can be used to gain a better understanding of how common time series features, alongside the frequency, shape, and correlation features from ROCKET, contribute to classification. Below is a demonstration of how the ROCKET transform matrix can be augmented with these 22 additional features.


In [ ]:
#handles correct formatting for the catch22 function
function apply_catch22(X::Array, channel = 1)
    return catch22(transpose(X[:,channel,:]))
end
#generate catch22 values per channel for all samples, return updated dataframe.
function gen_all(X::Array, df_to_add)
    df_new = copy(df_to_add)
    channel_count = size(X)[2]
    for i in 1:channel_count
        res = apply_catch22(X, i)
        setproperty!.(Ref(df_new), [string(la)*"_c"*string(i) for la in getnames(res)], eachrow(res.data));
    end
    return df_new
end

In [ ]:
#generate catch22
df_train_with_catch22 = gen_all(X_train, df_train) 
df_test_with_catch22 = gen_all(X_test, df_test)

In [ ]:
mach = regression_fitting(df_train_with_catch22[:,kers * 2:kers * 2 + 22 * n_dims], Y_train, 0.0, 1.0, 10, 5)

In [ ]:
#prediction
y_pred = predict_mode(mach,df_test_with_catch22[:,kers * 2:kers * 2 + 22 * n_dims])
y_train_pred = predict_mode(mach, df_train_with_catch22[:,kers * 2:kers * 2 + 22 * n_dims])

#confusion matrices
MLJ.confusion_matrix(y_pred, Y_test)
MLJ.confusion_matrix(y_train_pred, Y_train)


In [ ]:
sum(y_pred .==Y_test)/length(Y_test)

In [ ]:
mach = regression_fitting(df_train_with_catch22, Y_train, 0.0, 1.0, 10, 5)

In [ ]:
#prediction
y_pred = predict_mode(mach,df_test_with_catch22)
y_train_pred = predict_mode(mach, df_train_with_catch22)

#confusion matrices
MLJ.confusion_matrix(y_pred, Y_test)
#MLJ.confusion_matrix(y_train_pred, Y_train)
sum(y_pred .==Y_test)/length(Y_test)

## Example 2: Atrial Fibrillation

In this case, we take the Atrial Fibrillation dataset from http://www.timeseriesclassification.com/description.php?Dataset=AtrialFibrillation
This has a small training and testing dataset, with three classes and two channels.

In [ ]:
using CategoricalArrays
#import data
ds_string = "AtrialFibrillationDimension"
n_dims = 2
starts_at = 1
series_length = 640

X_train, Y_train = import_data(ds_string, n_dims, starts_at, true, true, series_length,641)
X_test, Y_test = import_data(ds_string, n_dims, starts_at, false, true, series_length,641)


In [ ]:

#Generate ROCKET kernels
kers = 5000
K = generate_kernels_mv(series_length,n_dims,kers);

#generate column names for each kernel -- for tracking relevant kernels later.
K_symbols = cat([Symbol(k.name,"max") for k in K], [Symbol(k.name,"ppv") for k in K],dims=1);

#transform data, store in dataframe with column labels for each kernel feature.
X_train_transform = Rocket_transform_mv(X_train,K)[1];
df_train = DataFrame(X_train_transform, K_symbols);
X_test_transform = Rocket_transform_mv(X_test,K)[1];
df_test = DataFrame(X_test_transform, K_symbols);


In [ ]:
#call regression fitting function on training data
mach = regression_fitting(df_train, Y_train, 0.0, 1.0, 10, 5,":")

In [ ]:

#prediction
y_pred = predict_mode(mach,df_test)
y_train_pred = predict_mode(mach, df_train)

#confusion matrices
MLJ.confusion_matrix(y_pred, Y_test)
MLJ.confusion_matrix(y_train_pred, Y_train)


In [ ]:
sum(y_pred .==Y_test)/length(Y_test) #random chance!

## Example 3: Pen Digits

In this case, we take the Pen Digits dataset from http://www.timeseriesclassification.com/description.php?Dataset=PenDigits
This has a large training and testing dataset, with ten classes and two channels.

In [ ]:
using CategoricalArrays
#import data
ds_string = "PenDigitsDimension"
n_dims = 2
starts_at = 1
series_length = 8

X_train, Y_train = import_data(ds_string, n_dims, starts_at, true, true, series_length,9)
X_test, Y_test = import_data(ds_string, n_dims, starts_at, false, true, series_length,9)


In [ ]:

#Generate ROCKET kernels
kers = 5000
K = generate_kernels_mv(series_length,n_dims,kers);

#generate column names for each kernel -- for tracking relevant kernels later.
K_symbols = cat([Symbol(k.name,"max") for k in K], [Symbol(k.name,"ppv") for k in K],dims=1);

#transform data, store in dataframe with column labels for each kernel feature.
X_train_transform = Rocket_transform_mv(X_train,K)[1];
df_train = DataFrame(X_train_transform, K_symbols);
X_test_transform = Rocket_transform_mv(X_test,K)[1];
df_test = DataFrame(X_test_transform, K_symbols);


In [ ]:
#call regression fitting function on training data
mach = regression_fitting(df_train, Y_train, 0.0, 1.0, 10, 5)

In [ ]:

#prediction
y_pred = predict_mode(mach,df_test)
y_train_pred = predict_mode(mach, df_train)

#confusion matrices
MLJ.confusion_matrix(y_pred, Y_test)
#MLJ.confusion_matrix(y_train_pred, Y_train)


In [ ]:
sum(y_pred .==Y_test)/length(Y_test)